<a href="https://colab.research.google.com/github/AkkalaGunavardhini/IIIt-FMML-Assignment-Lab/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?

 **A**.Let's explore how the accuracy of the validation set is affected when you increase or decrease the percentage of data allocated to it:

 **Increasing the Percentage of Validation Set:**

 **Positive Effects:** When you increase the percentage of data in the validation set, you typically have a smaller training set. This can be beneficial in the following ways:

 *Better* *Model* *Evaluation*:
 With a larger validation set, you can more accurately assess how well your model generalizes to unseen data. This can lead to a more reliable estimate of your model's performance.

 *Reduced* *Overfitting*: A smaller training set may help prevent overfitting since the model has less data to memorize, making it more likely to learn meaningful patterns.

 **Negative Effects:** However, there are downsides to having a larger validation set:

 *Reduced* *Training* *Data*: With a larger validation set, your training set becomes smaller. This can limit the amount of data available for training your model, potentially leading to underfitting if the dataset is already small.

 *Increased* *Variability*: The validation set's accuracy may become more variable because it's being evaluated on a smaller sample of data.
  
  **Positive Effects:** When you reduce the percentage of data in the validation set, you have a larger training set.




2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

  **A**.The size of the training and validation sets can affect how accurately we can predict the model's performance on the test set using the validation set in the following simple terms:

  **Larger** **Training** **Set**:
  
  If you have a bigger training set, the model has more data to learn from. This can help it perform better on both the validation and test sets. However, if the validation set is too small, it may not accurately represent how well the model will do on unseen data.

  **Larger** **Validation** **Set**:
  
   A larger validation set gives you a better estimate of how well your model generalizes to unseen data. This helps in predicting the test set performance more reliably. But, if the training set becomes too small, the model may not learn well enough.

  In essence, having a balance between a reasonable training set size and a representative validation set size is important. It ensures that your model learns well and that your validation results provide a good indication of how the model will perform on unseen data .


 3. what do you think good percentage to reserve for the validation set so that thest two factors are balanced?

   **A**. A common and balanced percentage to reserve for the validation set is around 20% of your total dataset.

  **Training Set (around 80%):** This gives your model enough data to learn meaningful patterns without overfitting. More training data helps your model become smarter.

  ***Validation Set (around 20%):***This is a reasonable chunk of data to estimate how well your model will perform on new, unseen data. It provides a good balance between model training and evaluation.

  The exact percentage can vary depending on your specific dataset and the problem you're solving, but around 80-20 is a good starting point for many situations.


Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

 **A**.Yes, averaging the validation accuracy across multiple splits can give more reliable and consistent results.

 **Imagine Flipping a Coin:**  If you flip a coin just once, you might get heads or tails, but you can't be sure if it's biased or fair based on just one flip. But if you flip it many times and calculate the average, you get a more accurate idea of the coin's fairness.

 **Similarly with Data Splits:** When you split your data into training and validation sets, there might be some randomness involved. Averaging the validation accuracy over multiple splits smooths out this randomness. It helps you get a more stable and trustworthy estimate of how well your model is likely to perform on unseen data.

 So, yes, averaging validation accuracy across multiple splits helps to get a more consistent and reliable assessment of your model's performance. It's like flipping the coin multiple times to be more confident about its fairness!

2. Does it give more accurate estimate of test accuracy?

   **A**.Yes, averaging validation accuracy across multiple splits can provide a more accurate estimate of test accuracy in simple terms.

   **Single Split:** If you just use one random split between training and validation data, your model's performance on the validation set might be influenced by luck. It's like taking one test and getting a good or bad score due to chance.

   **Multiple Splits:** When you average validation accuracy over multiple different splits, you're considering how your model performs in different scenarios. It's like taking the test several times under different conditions. Averaging these results gives you a more reliable estimate of how well your model is likely to perform on unseen data, which is like a final test.

   So, yes, averaging over multiple splits helps provide a more accurate estimate of how your model will likely do on the real test (the test set). It reduces the impact of luck or randomness in a single split.


3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

 **A**.The number of iterations in machine learning, particularly in training algorithms like gradient descent, can affect the quality of your model's estimate, but more iterations don't always guarantee a better estimate.

 **Few Iterations:** If you have too few iterations, your model may not have enough chances to learn and improve. It's like studying for an important test for only a few minutes—you probably won't do well.

 **Adequate Iterations:** There's an ideal number of iterations where your model gets a good balance between learning from the data and not overdoing it. It's like studying for a reasonable amount of time before a test.

 Too Many Iterations: If you have too many iterations, your model might start memorizing the training data (overfitting) rather than learning useful patterns. It's like studying for the test for hours and hours, to the point where you're just memorizing answers without understanding the material.


4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

  **A**.Increasing the number of iterations can help to some extent when you have a very small training or validation dataset, but it's not a complete solution.

 Small Training Dataset: If your training dataset is tiny, more iterations can help your model learn better from the limited data, but it won't magically make up for the lack of diverse examples. Think of it like trying to learn a lot from a very short book—you can read it many times, but it won't contain new information. Small Validation Dataset: If your validation dataset is small, more iterations can give a better estimate of your model's performance, but it won't eliminate the limitations of having a small sample to judge your model's ability.
